# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
!pip install nltk

In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
# import libraries
from sqlalchemy import create_engine 
import nltk
import pandas as pd
import sqlite3
import pickle

import re
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [4]:
# load data from database
engine = create_engine('sqlite:///Disaster_Analysis.db')
df = pd.read_sql_table('Messages_with_Cats', engine)

# Choose the X and Y (feature and target variables) for ML Model
X = df['message']
Y = df.iloc[:,4:]


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    
    '''
    INPUT 
        text: Text to be processed   
    OUTPUT
        Returns an array of words after being processed; by being tokenized, lower cased, stripped, and lemmatized
    '''
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    
    '''
    Define a custom estimator that extract the feature from a sentence checking if the text is starting with a verb
    
    '''

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [7]:
def model_pipeline():
    
    pipeline = Pipeline([
            ('features', FeatureUnion([

                ('text_pipeline', Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer())
                ]))
            ])),
        
            ('clf', MultiOutputClassifier(KNeighborsClassifier()))
        ])
    return pipeline

In [8]:
def model_pipeline_improved():
    
    pipeline = Pipeline([
            ('features', FeatureUnion([

                ('text_pipeline', Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer())
                ])),

                ('starting_verb', StartingVerbExtractor())
            ])),

            ('clf', MultiOutputClassifier(KNeighborsClassifier()))
        ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# Define the model and fit the training dataset
model = model_pipeline()
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
# Predict from trained model using the test dataset
y_pred = model.predict(X_test)

In [16]:
def classificaion_report_iteration(true_y,pred_y,col_names):
    
    '''
    INPUT 
        true_y: Test dataset true results
        pred_y: Predicted results from the test_X in the testing dataset
        col_names: Column Names for each category in order to print its scores
    OUTPUT
        Prints out the scores of each category using the pre-defined function 
        classification_report along which the category name
    '''
    
    for i in range(pred_y[0,:].size):
        print(col_names[i])
        print(classification_report(true_y[i,:],pred_y[i,:]))

In [17]:
classificaion_report_iteration(y_test.values, y_pred,Y.columns.values)

related
             precision    recall  f1-score   support

          0       0.97      0.88      0.92        32
          1       0.43      0.75      0.55         4

avg / total       0.91      0.86      0.88        36

request
             precision    recall  f1-score   support

          0       1.00      0.97      0.99        36
          1       0.00      0.00      0.00         0

avg / total       1.00      0.97      0.99        36

offer
             precision    recall  f1-score   support

          0       0.83      1.00      0.91        29
          1       1.00      0.14      0.25         7

avg / total       0.86      0.83      0.78        36

aid_related
             precision    recall  f1-score   support

          0       0.97      1.00      0.99        35
          1       0.00      0.00      0.00         1

avg / total       0.95      0.97      0.96        36

medical_help
             precision    recall  f1-score   support

          0       0.83      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
        'clf__estimator__n_neighbors': [3, 5]
    }

# Create new model with same pipeline and apply grid search on it
model2 = model_pipeline()
cv = GridSearchCV(model2, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, y_train)
#cv.get_params()

In [ ]:
y_pred2 = cv.predict(X_test)

In [ ]:
classificaion_report_iteration(y_test.values, y_pred2,Y.columns.values)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
model_improved = model_pipeline_improved()
model_improved.fit(X_train, y_train)

In [ ]:
y_pred3 = model_improved.predict(X_test)

In [ ]:
classificaion_report_iteration(y_test.values, y_pred3,Y.columns.values)

### 9. Export your model as a pickle file

In [ ]:
file = open(model_filepath, 'wb')
pickle.dump(model, file)
file.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.